In [152]:
import sys

sys.path.insert(0, "/Users/maxdumas/cornell/spec-project/flow/")

In [153]:
import numpy as np

In [154]:
import flow
from flow.networks.ring import RingNetwork
from flow.controllers.car_following_models import IDMController
from flow.controllers.routing_controllers import ContinuousRouter
from flow.envs.ring.lane_change_accel import LaneChangeAccelEnv, ADDITIONAL_ENV_PARAMS
from flow.core.experiment import Experiment
from flow.networks.ring import ADDITIONAL_NET_PARAMS
from flow.core.params import (
    NetParams,
    EnvParams,
    SumoParams,
    TrafficLightParams,
    InitialConfig,
    VehicleParams,
)


In [155]:
from flow.controllers.base_lane_changing_controller import BaseLaneChangeController

# We want the ambulance to always prefer the left lane, so we create this
# lane-changing controller that makes them always enter the left lane
class LeftLaneController(BaseLaneChangeController):
    """A lane-changing model used to move vehicles into the left lane."""

    def get_lane_change_action(self, env):
        current_lane = env.k.vehicle.get_lane(self.veh_id)
        if current_lane < 4:
            return 1
        else:
            return 0


# We want to have vehicles merge to the right if possible when they detect that
# the ambulance is coming up behind them.
class YieldToEmergencyVehicleLaneChangeController(BaseLaneChangeController):
    def get_lane_change_action(self, env):
        current_lane = env.k.vehicle.get_lane(self.veh_id)
        # If any follower is an ambulance, merge right
        follower_ids = env.k.vehicle.get_lane_followers(self.veh_id)
        if any(follower_id.startswith("ambulance") for follower_id in follower_ids):
            return -1
        else:
            return np.random.choice([-1, 0, 1])


In [156]:
class YieldToEmergencyVehicleController(IDMController):
    def get_accel(self, env):
        current_lane = env.k.vehicle.get_lane(self.veh_id)
        v = env.k.vehicle.get_speed(self.veh_id)
        x = env.k.vehicle.get_x_by_id(self.veh_id)
        follower_ids = env.k.vehicle.get_lane_followers(self.veh_id)
        leader_ids = env.k.vehicle.get_lane_leaders(self.veh_id)
        # If any follower or leader is an ambulance, slow down.
        for follower_id in follower_ids + leader_ids:
            if follower_id.startswith("ambulance"):
                ambulance_dist = env.k.vehicle
                ambulance_lane = env.k.vehicle.get_lane(follower_id)
                ambulance_v = env.k.vehicle.get_speed(follower_id)
                ambulance_x = env.k.vehicle.get_x_by_id(follower_id)
                print(abs(ambulance_x - x))
                # If the ambulance is in our lane, maintain speed and focus on merging right
                if ambulance_lane == current_lane or 200 < abs(ambulance_x - x) < 1000:
                    return IDMController.get_accel(self, env)
                else:
                    return -0.5
                    # Decelerate so that we reach 50% of ambulance speed in 10 timesteps
                    # return 0.1 * (ambulance_v * 0.5 - v)

        

        return IDMController.get_accel(self, env)


In [157]:
vehicles = VehicleParams()
vehicles.add(
    "ambulance",
    acceleration_controller=(IDMController, { "v0": 70 }),
    lane_change_controller=(LeftLaneController, {}),
    routing_controller=(ContinuousRouter, {}),
    num_vehicles=1,
    color="red",
)
vehicles.add(
    "human",
    acceleration_controller=(YieldToEmergencyVehicleController, {}),
    lane_change_controller=(YieldToEmergencyVehicleLaneChangeController, {"lane_change_params": {"min_gap": 0.0}}),
    routing_controller=(ContinuousRouter, {}),
    num_vehicles=100,
)


In [158]:
exp = Experiment(
    dict(
        exp_tag="ambulance",
        env_name=LaneChangeAccelEnv,
        network=RingNetwork,
        simulator="traci",
        sim=SumoParams(sim_step=0.1, render=True),
        env=EnvParams(horizon=3000, additional_params=ADDITIONAL_ENV_PARAMS),
        net=NetParams(additional_params={**ADDITIONAL_NET_PARAMS, "lanes": 4, "length": 1200}),
        veh=vehicles,
        initial=InitialConfig(spacing="uniform", perturbation=1),
        tls=TrafficLightParams(),
    )
)

# run the sumo simulation
_ = exp.run(1)


INITIALIZING traci
0.0
0.0
0.0
45.84190845156576
1187.1423421998009
0.009877247592675909
0.009877247592675909
0.009877247592675909
45.841910750772726
1141.7029013047984
1187.1418472037217
0.029629339789911453
0.029629339789911453
0.029629339789911453
45.841915581367445
1141.702912116823
1187.1265268679888
0.05925375847348897
0.05925375847348897
0.05925375847348897
45.84192318655083
1187.096946328082
0.09874787052613297
0.09874787052613297
0.09874787052613297
45.841933820477834
1187.0668834966564
0.1481089279690558
0.1481089279690558
0.1481089279690558
45.84194774816298
1187.0363017562474
0.20733406810611055
0.20733406810611055
0.20733406810611055
45.84196524535751
1187.0051639095702
0.27642031367465286
0.27642031367465286
0.27642031367465286
45.84198659839824
1141.7030052779885
1186.973432288198
0.3553645730032095
0.3553645730032095
0.3553645730032095
45.842012104028264
1141.7030367120499
1186.9268921580165
0.44416364017604587
0.44416364017604587
0.44416364017604587
45.84204206918937
1

FatalTraCIError: connection closed by SUMO